<a href="https://colab.research.google.com/github/satisdrive/privacy_grade_predictor/blob/master/tosdr_privacy_grade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install ktrain on Google Colab
!pip3 install ktrain

In [2]:
# import ktrain and the ktrain.text modules
import ktrain
from ktrain import text


In [3]:
ktrain.__version__

'0.25.4'

In [4]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv
Saving val.csv to val.csv


In [5]:
import pandas as pd
import io

train_df = pd.read_csv(io.StringIO(uploaded['train.csv'].decode('utf-8')))
val_df = pd.read_csv(io.StringIO(uploaded['val.csv'].decode('utf-8')))
test_df = pd.read_csv(io.StringIO(uploaded['test.csv'].decode('utf-8')))


In [18]:
import numpy as np

MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, class_names=['bad','blocker','good','neutral'])
trn = t.preprocess_train(np.array(train_df['text']), np.array(train_df['point']))
val = t.preprocess_train(np.array(val_df['text']), np.array(val_df['point']))

preprocessing train...
language: en
train sequence lengths:
	mean : 38
	95percentile : 92
	99percentile : 118


/usr/local/lib/python3.6/dist-packages/ktrain/utils.py:589: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing train...
language: en
train sequence lengths:
	mean : 40
	95percentile : 91
	99percentile : 190


Is Multi-Label? False


In [19]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)
learner.fit_onecycle(5e-5, 5)



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/5
62/62 [==============================] - 35s 463ms/step - loss: 1.3085 - accuracy: 0.3407 - val_loss: 1.1582 - val_accuracy: 0.4420
Epoch 2/5
62/62 [==============================] - 27s 434ms/step - loss: 1.1031 - accuracy: 0.4915 - val_loss: 1.0423 - val_accuracy: 0.5746
Epoch 3/5
62/62 [==============================] - 27s 430ms/step - loss: 0.7018 - accuracy: 0.7750 - val_loss: 1.0749 - val_accuracy: 0.5856
Epoch 4/5
62/62 [==============================] - 27s 436ms/step - loss: 0.4660 - accuracy: 0.8442 - val_loss: 1.1589 - val_accuracy: 0.5746
Epoch 5/5
62/62 [==============================] - 27s 434ms/step - loss: 0.2394 - accuracy: 0.9351 - val_loss: 1.0635 - val_accuracy: 0.6298


In [20]:
learner.validate(class_names=t.get_classes()) # class_names must be string values

              precision    recall  f1-score   support

         bad       0.72      0.61      0.66        79
     blocker       0.00      0.00      0.00         9
        good       0.33      0.54      0.41        24
     neutral       0.71      0.77      0.74        69

    accuracy                           0.63       181
   macro avg       0.44      0.48      0.45       181
weighted avg       0.63      0.63      0.62       181



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[48,  0, 17, 14],
       [ 3,  0,  3,  3],
       [ 6,  0, 13,  5],
       [10,  0,  6, 53]])

## How to Use Our Trained BERT Model

We can call the `learner.get_predictor` method to obtain a Predictor object capable of making predictions on new raw data.

In [21]:
predictor = ktrain.get_predictor(learner.model, t)

In [22]:
predictor.get_classes()

['bad', 'blocker', 'good', 'neutral']

In [23]:
predictor.predict('when youre not signed in to a google account we store the information we collect with unique identifiers tied to the browser application or device youre using.',return_proba=True)

array([0.3545409 , 0.02966522, 0.40360904, 0.2121848 ], dtype=float32)

In [24]:
for i in range(10):
  print('Pred: {0} : True: {1} : {2}'.format(predictor.predict(test_df['text'][i]),test_df['point'][i],test_df['text'][i]))

Pred: neutral : True: neutral : we always indicate the date the last changes were published
Pred: neutral : True: good : none
Pred: good : True: bad : when youre not signed in to a google account we store the information we collect with unique identifiers tied to the browser application or device youre using.
Pred: neutral : True: neutral : when we detect spam malware illegal content and other forms of abuse on our systems in violation of our policies we may disable your account or take other appropriate action.
Pred: bad : True: bad : we collect information about your location when you use our services which helps us offer features like driving directions for your weekend getaway or showtimes for movies playing near you.p p
Pred: good : True: neutral : you can also configure your browser to block all cookies from a specific domain or all domains. but remember that our services rely on cookies to function properly for things like remembering your language preferences.
Pred: good : True

In [ ]:
# we predicted the correct label
print(test_b.target_names[test_b.target[0]])

sci.med


The `predictor.save` and `ktrain.load_predictor` methods can be used to save the Predictor object to disk and reload it at a later time to make predictions on new data.

In [25]:
# let's save the predictor for later use
predictor.save('privacy_grade_predictor')

In [26]:
# reload the predictor
reloaded_predictor = ktrain.load_predictor('privacy_grade_predictor')

In [27]:
for i in range(10):
  print('Pred: {0} : True: {1} : {2}'.format(reloaded_predictor.predict(test_df['text'][i]),test_df['point'][i],test_df['text'][i]))

Pred: neutral : True: neutral : we always indicate the date the last changes were published
Pred: neutral : True: good : none
Pred: good : True: bad : when youre not signed in to a google account we store the information we collect with unique identifiers tied to the browser application or device youre using.
Pred: neutral : True: neutral : when we detect spam malware illegal content and other forms of abuse on our systems in violation of our policies we may disable your account or take other appropriate action.
Pred: bad : True: bad : we collect information about your location when you use our services which helps us offer features like driving directions for your weekend getaway or showtimes for movies playing near you.p p
Pred: good : True: neutral : you can also configure your browser to block all cookies from a specific domain or all domains. but remember that our services rely on cookies to function properly for things like remembering your language preferences.
Pred: good : True